# Enunciado


2. Usando o protocolo  OT construído na questão anterior
    1. Implemente o protocolo $\,\mathsf{sVOLE}\;$ (“subset vectorial oblivious linear evaluation”)
    2. Usando $\,\mathsf{sVOLE}\,$   implemente um protótipo de um protocolo  ZK-sVOLE usando equações polinomiais do 2º grau aleatoriamente geradas.

----------

# PARTE A

## Definição de variáveis

Para definir o protocolo vamos começar por fixar os parâmetros, $\,p\,$, $\,k\,$ e $\,\ell\,$, que determinam a estrutura algébrica onde o protocolo se desenvolve.

In [1]:
p = 3 
k = 5  
l = 8 
lambda_security = 128 
N = 2 * k

print_messagens_geradas = True

## Estruturas algébricas

In [2]:
F = GF(p)
R.<x> = PolynomialRing(F)
f = R.irreducible_element(k)
E.<a> = GF(p^k, name='a', modulus=f)  

def verifica_base(Z, E):
    F = GF(E.characteristic())
    V = VectorSpace(F, E.degree())  # aqui garantidamente é um VectorSpace
    for j in range(len(Z)):
        subZ = Z[:j] + Z[j+1:]
        mat = matrix(F, [V(E(z)) for z in subZ])
        if mat.rank() < E.degree():
            return False
    return True
    
def gerarZ(E, N):
    F = GF(E.characteristic())
    k = E.degree()
    V = VectorSpace(F, k)  # <-- aqui está garantido que é mesmo um VectorSpace
    tentativas = 0
    while True:
        tentativas += 1
        Z = set()
        while len(Z) < N:
            v = V.random_element()
            if not v.is_zero():
                Z.add(E(v))  # converte vetor para elemento de E
        Z = list(Z)
        if verifica_base(Z, E):
            print(f"Conjunto Z encontrado após {tentativas} tentativas.")
            return Z

Z = gerarZ(E,N)

Conjunto Z encontrado após 1 tentativas.


Esta estrutura algébrica fornece a base para o protocolo OLE, que permite a avaliação linear de forma “oblivious”, sendo uma primitiva fundamental em construção de protocolos criptográficos como OT.

## Oblivious transfer da questão anterior:

In [3]:
import random
import hashlib

def bernoulli(epsilon, n=53):
    """
    Gera uma amostra de Bernoulli B(epsilon) usando a construção de Lebesgue.
    - epsilon: parâmetro da distribuição de Bernoulli (0 < epsilon < 1)
    - n: número de bits para a precisão (default: 53, precisão de um double)
    """
    # Gera a string de bits aleatórios {0,1}^n
    w = [random.randint(0, 1) for _ in range(1, n+1)]
    
    # Calcula o racional de Lebesgue
    w_hat = sum(w[i-1] * 2^(-i) for i in range(1,n+1))
    
    return 1 if w_hat <= epsilon else 0

def bernoulli_lambda(epsilon, n=53, lambda_ = lambda_security):
    return [bernoulli(epsilon,n) for _ in range(lambda_)]

def vector_to_bytes(vector, p):
    byte_length = (p.bit_length() + 7) // 8  # Calcula bytes suficientes para representar elementos de F
    return b''.join(
        int(x).to_bytes(byte_length, 'big', signed=False)  # Garante representação não assinada
    for x in vector)

def bytes_to_vector(byte_stream, p, l):
    byte_length = (p.bit_length() + 7) // 8
    elementos = []
    for i in range(0, len(byte_stream), byte_length):
        chunk = byte_stream[i:i+byte_length]
        if not chunk:
            elementos.append(F(0))
            continue
        valor = int.from_bytes(chunk, 'big') % p
        elementos.append(F(valor))
    return vector(F, elementos[:l])

def converter_bytes_para_mensagens(byte_stream_list, p, tamanho, N):
    byte_length = (p.bit_length() + 7) // 8  
    mensagens = {}
    for n in range(1, N + 1):
        if n-1 < len(byte_stream_list) and byte_stream_list[n-1] is not None:
            byte_stream = byte_stream_list[n-1]
            elementos = []
            # Processa cada elemento em chunks de 'byte_length' bytes
            for i in range(0, len(byte_stream), byte_length):
                chunk = byte_stream[i:i+byte_length]
                if not chunk:
                    elementos.append(F(0))
                    continue
                valor = int.from_bytes(chunk, 'big') % p
                elementos.append(F(valor))
            # Preenche com zeros se faltarem elementos
            while len(elementos) < tamanho:
                elementos.append(F(0))
            mensagens[n] = vector(F, elementos[:tamanho])
        else:
            mensagens[n] = vector(F, [F(0)] * tamanho)  # Preenche com zeros
    return mensagens

def xof(seed: bytes, nbytes: int):
    shake = hashlib.shake_128()
    shake.update(seed)
    return shake.digest(nbytes)  # retorna nbytes de saída
    
def xof_bits(seed: bytes, nbits: int):
    nbytes = (nbits + 7) // 8
    output = xof(seed, nbytes)
    bits = []
    for byte in output:
        for i in range(8):
            bits.append((byte >> (7 - i)) & 1)
            if len(bits) == nbits:
                return bits

In [4]:
class Sender:
    def __init__(self, alpha, l, xof_name="shake128"):
        self.alpha = alpha
        self.l = l
        self.xof_name = xof_name
        self.seed = f"{alpha}:{lambda_security}".encode()  # usa α e ℓ como seed
        self.criterion = None
        self.pks = []
        self.N = N  
        self.mensagens = None
        self.r = []
        self.criptogramas = []
        self.t = 100

    def get_vector_ai_ui(self, i, lambda_):
        seed_i = self.seed + f":{i+1}".encode()
        bits = xof_bits(seed_i, lambda_ + 1)
        a_i = bits[:lambda_]
        u_i = bits[lambda_]
        return a_i, u_i

    def get_criterion_sequence(self, lambda_):
        self.criterion = [self.get_vector_ai_ui(i, lambda_) for i in range(self.l)]

    def receive_and_verifypks(self, pks):
        self.pks = pks
        for i in range(self.l):
            soma = sum(pks[i][klinha] for klinha in range(self.N)) % 2
            _, u_i = self.criterion[i]
            if soma != u_i:
                print(f"FALHA na linha i={i}: soma={soma}, u_i={u_i}")
                print(f"pks[{i}] = {pks[i]}")
                print(f"u_{i} = {u_i}")
                print("------------------")
                return False
        print("Verificação OK: todas as somas coincidem com u_i")
        print("------------------")
        return True

    def gerar_r(self, delta, p=0.1):
        while True:
            r = [bernoulli(p) for _ in range(self.l)]
            if sum(r) <= delta:
                self.r = r
                return

    def calcular_criptogramas(self):
        self.criptogramas = []
        for _ in range(self.t):
            self.gerar_r(128)
            a = [0] * lambda_security  # Agora em F_p
            for i in range(self.l):
                if self.r[i] == 1:
                    a_i, _ = self.criterion[i]
                    a = [(a[j] + a_i[j]) % p for j in range(lambda_security)]
            criptogramas_k = []
            for k in range(1,N+1):
                msg_vector = self.mensagens[k]  # msg_vector é um vetor em F_p^ℓ
                c_k = []
                for elemento in msg_vector:
                    soma = sum(self.r[i] * self.pks[i][k-1] for i in range(self.l)) % 2
                    c_k.append((elemento + soma) % p)
                criptogramas_k.append(c_k)
            self.criptogramas.append((a, criptogramas_k))
            
    def print_info(self):
        print("self.alpha: ",self.alpha)
        print("self.l: ",self.l)
        print("self.xof_name: ",self.xof_name)
        print("self.seed: ",self.seed)
        print("self.criterion: ",self.criterion)

In [5]:
class Receiver:
    def __init__(self, eps, b, lambda_, n_mensagens):
        self.alpha = None
        self.l = None
        self.eps = eps
        self.secrets = []
        self.criterion = None
        self.N = n_mensagens
        self.t = None
        self.b = b - 1

    def receive_alpha_l(self, alpha, l):
        self.alpha = alpha
        self.l = l
        self.seed = f"{alpha}:{lambda_security}".encode()
        self.criterion = self.get_criterion_sequence(lambda_security)

    def get_vector_ai_ui(self, i, lambda_):
        seed_i = self.seed + f":{i+1}".encode()
        bits = xof_bits(seed_i, lambda_ + 1)
        a_i = bits[:lambda_]
        u_i = bits[lambda_]
        return a_i, u_i

    def get_criterion_sequence(self, lambda_):
        if self.l is None:
            raise ValueError("self.l não está definido")
        print(f"Seed usado: {self.seed}")
        criterion = [self.get_vector_ai_ui(i, lambda_) for i in range(self.l)]
        return criterion

    def generate_N_secrets(self):
        self.secrets = []
        for k in range(self.N):
            if k == self.b:
                self.secrets.append(None)  # s_b = ⊥
            else:
                s_k = bernoulli_lambda(self.eps, lambda_security)
                self.secrets.append(s_k)

    def generate_pks(self):
        self.t = [[0 for _ in range(self.N)] for _ in range(self.l)]  
        for i in range(self.l):
            a_i, u_i = self.criterion[i]
            soma = 0
            for k in range(self.N):
                if k != self.b:
                    s_k = self.secrets[k]
                    dot = sum([a_i[j] * s_k[j] for j in range(lambda_security)]) % 2
                    e = bernoulli(self.eps)
                    t_ik = (dot + e) % 2
                    self.t[i][k] = t_ik
                    soma = (soma + t_ik) % 2
                else:
                    self.t[i][k] = None
            self.t[i][self.b] = (u_i - soma) % 2
        return self.t

    def recuperar_mensagens(self, a, c):
        mensagens_recuperadas = []
        for k in range(N):
            if k == self.b:
                mensagens_recuperadas.append(None)
            else:
                s_k = self.secrets[k]
                # Garantir que a e s_k estão no mesmo campo
                a_dot_s = sum(F(a[j]) * F(s_k[j]) for j in range(lambda_security)) % p
                msg_recuperada = [F((c[k][i] + a_dot_s) % p) for i in range(len(c[k]))]
                mensagens_recuperadas.append(msg_recuperada)
        return mensagens_recuperadas

    def recuperar_mensagens_maioritarias(self, lista_criptogramas):
        t = len(lista_criptogramas)
        resultados_por_k = [[] for _ in range(self.N)]
        for i in range(t):
            a, c = lista_criptogramas[i]
            mks = self.recuperar_mensagens(a, c)
            for k in range(self.N):
                resultados_por_k[k].append(tuple(mks[k]) if mks[k] is not None else None)
        mensagens_finais = []
        for k in range(self.N):
            if k == self.b:
                mensagens_finais.append(None)
            else:
                contagem = {}
                for msg in resultados_por_k[k]:
                    if msg is not None:
                        # Ignora vetores totalmente zero
                        if all(x == 0 for x in msg):
                            continue
                        msg_key = tuple(msg)
                        contagem[msg_key] = contagem.get(msg_key, 0) + 1
                if contagem:
                    mensagem_mais_votada = max(contagem.items(), key=lambda x: x[1])[0]
                    byte_msg = vector_to_bytes(mensagem_mais_votada, p)
                else:
                    byte_msg = None  # Todos eram vetores de zeros
                mensagens_finais.append(byte_msg)
        return mensagens_finais

## Protocolo sVOLE

O protocolo OLE  simples tem simultaneamente uma questão de segurança e uma questão de eficiência ambas com a mesma proveniência: o número de elementos $\,p^k\,$ do corpo $E$.

- A questão de segurança resulta de, para existir capacidade de verificar o conhecimento de $\,x\in E\,$ ambos os “tags”  $\,\mathsf{m}\,,\,\mathsf{q}\in E\,$ têm de ser suficientemente grandes. Assim o tamanho dessas “tags”, que no protocolo OLE  é  $\,|p|\times k\,$, exige um valor mínimo razoável ($\sim 128\,$bits, ou superior).
- Ainda, em termos de segurança, o tamanho da chave global $\,\Delta\,$ tem de ser sufientemente grande para proteger o protocolo de ataques. Como $\,\Delta\,$ é um valor gerado de $\,E\,$ o factor “segurança de $\Delta$”  exige também um valor mínimo para $\,p^k\,$.
- A questão de eficiência resulta do facto de o número $\,N\,$ de mensagens usadas no protocolo OT  e usadas nas somas que calculam os “tags” ser também $\,p^k\,$. A eficiência exige que $\,N\,$ não possa ser muito grande.


Para criar uma solução que responda  simultaneamente a todas estas questões, a partir de um factor de segurança $\,\lambda\,$ e vamos seguir os seguintes objectivos

1. Os “tags” $\,\mathsf{m}\,$e $\,\mathsf{q}\,$ passam a ser vetores com $\,\ell\,$ componentes em $\,E\,$; como  elementos de $\,E^\ell\,$ o  o seu tamanho é substancialmente aumentado para  $\,|p|\times k\times \ell\,$;  se a dimensão $\,\ell\,$ for suficientemente grande, mesmo com $|p|$ e $\,k\,$ relativamente pequenos 
2. Os índices $\,z\,$ que identificam os elementos $\,t_z\,$ usados no protocolo OT   formam um subconjunto  $\,\mathcal{Z}\,\equiv\,\{z_1,z_2,\cdots\,z_N\}\,\subset E\,$ com $\,N  = \mathsf{poly}(\lambda)$  elementos.  
    O valor de $\,N\,$ e os elementos $\,\{z_n\}_{n\in[N]}\,$ são escolhidos de tal forma que, para todo $\,n\in [N]\,$ o conjunto $\,\mathcal{Z}\setminus\{z_n\}\,$ é uma base de $\,\mathbb{F}_{p^k}\,$ visto como espaço vectorial sobre $\,\mathbb{F}_p$.  
    Concretamente, cada $\,a\in E\,$ é uma combinação linear, com coeficientes em $\,F\,,$ dos elementos de $\,\mathcal{Z}\,$ e , cada $\,z_n\in\mathcal{Z}\,$ é uma combinação linear dos elementos de $\,\mathcal{Z}\setminus\{z_n\}\,$.
3. Os valores a autenticar $\,\overline{x}\,=\,\langle x_1, x_2,\cdots,x_\ell\rangle\,$ passam a ser também vectores de domensão $\,\ell\,$; isto é $\,\overline{x}\in F^{\ell}$.
4. A chave global $\,\Delta\,$ é escolhida, em cada instância do protocolo, dentro do  conjunto $\,\mathcal{Z}\,$ através da seleção de um índice $\,j\gets [N]\,$, e depois escolhendo $\,\Delta \equiv z_j\,$.
                                              $\Delta\,\gets\,\{\,\vartheta\,j\gets [N]\,\centerdot\, z_j\,\}$
5. Os valores $\,t_z\in F\,\,$ do protocolo OLE são substituídos por vectores; assim, para todo $\,i\in[\ell]\,$,  define-se um vector  $\,\overline{t}_i = \{t_{i,n}\}_{n\in[N]}\in\,F^\ell\,$ em que, 
        1. para $\,n>1\,$ , tem-se $\,t_{i,n}\,\gets\,F\,$ é gerado pseudo-aleatoriamente
        2. para  $\,n=1\,$  tem-se  $t_{i,1}\,\gets\, x_i - \sum_{n=2}^N\,t_{i,n}$.
    Isto assegura que, para todo $\,i\in[\ell]\,$, se verifica $\,x_i \,=\,\sum_{n\in[N]}\,t_{i,n}$



6. Ambos os agentes, Prover e Verifier  conhecem os parâmetros  $\,p,k,\ell\,$ e o parâmetro de segurança $\,\lambda\,$. Conhecem também $\,\mathcal{Z} \subset \mathbb{F}_{p^k}\,$ e a sua cardinalidade $\,N=O(\mathsf{poly}(\lambda))\,$, como especificámos atrás.

### Prover sVOLE

Este agente conhece um vector $\,\overline{x}\in F^\ell\,$ , que é sua informação privada, e procede do seguinte modo 

a. para todo $\,i\in [\ell]$ \
   i. Gera o vector $\,\overline{t}_i\,$  como está indicado no ponto 5. acima, \
   ii. Disponibiliza $\,N\,$ mensagens $\,\{m_{i,n} \gets \sigma(t_{i,n})\}_{n\in [N]}\,$   para transferência num protocolo “oblivious transfer”  $\,{N\choose{N-1}}$ . \
   iii. Calcula o “tag”  $\;\mathsf{m}_i\,\gets\,\sum_{n\in[N]}\,(-z_n)\cdot t_{i,n}\,$.

b. Agrega num vector $\;\overline{\mathsf{m}}\in E^\ell\;$ os vários “tags”  $\,\{\mathsf{m}_i\}_{i\in [\ell]}$

In [6]:
class Prover_sVOLE:
    def __init__(self,ell):
        self.ell = ell
        self.vectorX = vector(F, [F.random_element() for _ in range(self.ell)])
        self.t_matrix = {}
        self.messages = []
        self.sender = Sender(alpha="alpha123", l = self.ell)
        self.tag = None

    #passo a.i
    def gerar_tz(self):
        for i in range(len(self.vectorX)):
            t_i = {}
            # Gere N-1 elementos aleatórios (de 2 a N)
            for n in range(2, N + 1):
                t_i[n] = F.random_element()
            # Calcule t_i[1] para garantir que a soma seja x[i]
            t_i[1] = self.vectorX[i] - sum(t_i.values())
            # Ordene as chaves para garantir a ordem
            self.t_matrix[i] = {k: t_i[k] for k in sorted(t_i.keys())}

    def mostrar_estrutura(self):
        print(f"Vetor privado x: {self.vectorX}")
        print("\nMatriz t_{i,n}:")
        for i in range(min(3, len(self.vectorX))):  # Mostra apenas as 3 primeiras linhas para exemplo
            print(f"i={i}: {self.t_matrix.get(i, 'Não gerado')}")

    #ponto a.ii
    def gen_messages(self):
        self.messages = {}
        for n in range(1, N + 1):
            # Cria um vetor com os elementos t_{i,n} para todos i
            msg_vector = [self.t_matrix[i][n] for i in range(self.ell)]
            # Converte para bytes corretamente
            byte_msg = vector_to_bytes(vector(F, msg_vector), p)
            self.messages[n] = byte_msg
        
        if print_messagens_geradas:
            print("Mensagens geradas:")
            for n in range(1, N + 1):
                print(f"{n}: {self.messages[n]}")
            print("------------------")

    # Função sigma que codigica um elemento de F como string de bits
    def _sigma(self, x):
        bit_length = (p - 1).bit_length()
        
        if x in F: 
            return format(int(x), f'0{bit_length}b')
        elif isinstance(x, list) and all(y in F for y in x):
            return ''.join([format(int(y), f'0{bit_length}b') for y in x])
        else:
            raise ValueError("x deve ser um elemento de F ou uma lista de elementos de F")

    #ponto a.iii / b
    def gerar_tag(self):
        self.tag = []
        for i in range(len(self.vectorX)):
            soma = E(0)
            for n in range(1, N + 1):
                z_n = Z[n - 1]
                t_in = self.t_matrix[i][n]
                soma += (-z_n) * E(t_in)
            self.tag.append(soma)
        print("Tag calculado:", self.tag)
        print("------------------")
            
prover_sVOLE = Prover_sVOLE(l)
prover_sVOLE.gerar_tz()
prover_sVOLE.mostrar_estrutura()
prover_sVOLE.gen_messages()

from sage.modules.vector_modn_dense import Vector_modn_dense

def dicionario_para_vetores(messages_dict, p, tamanho):
    byte_length = (p.bit_length() + 7) // 8
    vetores = {}
    for n in messages_dict:
        byte_stream = messages_dict[n]
        elementos = []
        for i in range(0, len(byte_stream), byte_length):
            chunk = byte_stream[i:i+byte_length]
            valor = int.from_bytes(chunk, 'big') % p
            elementos.append(F(valor))
        # Preenche com zeros se faltarem elementos
        while len(elementos) < tamanho:
            elementos.append(F(0))
        vetores[n] = vector(F, elementos[:tamanho])
    return vetores

# Uso:
vetores_OT = dicionario_para_vetores(prover_sVOLE.messages, p, l)
print("vetores_OT: ",vetores_OT)
prover_sVOLE.sender.mensagens = vetores_OT
prover_sVOLE.gerar_tag()

Vetor privado x: (1, 1, 0, 2, 1, 0, 0, 0)

Matriz t_{i,n}:
i=0: {1: 2, 2: 1, 3: 1, 4: 2, 5: 1, 6: 2, 7: 1, 8: 1, 9: 0, 10: 2}
i=1: {1: 2, 2: 0, 3: 0, 4: 1, 5: 0, 6: 2, 7: 1, 8: 2, 9: 2, 10: 0}
i=2: {1: 0, 2: 1, 3: 0, 4: 2, 5: 1, 6: 2, 7: 1, 8: 1, 9: 2, 10: 2}
Mensagens geradas:
1: b'\x02\x02\x00\x00\x02\x00\x01\x01'
2: b'\x01\x00\x01\x01\x00\x00\x02\x01'
3: b'\x01\x00\x00\x00\x00\x01\x02\x00'
4: b'\x02\x01\x02\x01\x02\x00\x00\x00'
5: b'\x01\x00\x01\x00\x00\x01\x02\x01'
6: b'\x02\x02\x02\x02\x01\x01\x02\x02'
7: b'\x01\x01\x01\x02\x00\x02\x01\x01'
8: b'\x01\x02\x01\x02\x02\x01\x01\x02'
9: b'\x00\x02\x02\x02\x00\x01\x01\x01'
10: b'\x02\x00\x02\x01\x00\x02\x00\x00'
------------------
vetores_OT:  {1: (2, 2, 0, 0, 2, 0, 1, 1), 2: (1, 0, 1, 1, 0, 0, 2, 1), 3: (1, 0, 0, 0, 0, 1, 2, 0), 4: (2, 1, 2, 1, 2, 0, 0, 0), 5: (1, 0, 1, 0, 0, 1, 2, 1), 6: (2, 2, 2, 2, 1, 1, 2, 2), 7: (1, 1, 1, 2, 0, 2, 1, 1), 8: (1, 2, 1, 2, 2, 1, 1, 2), 9: (0, 2, 2, 2, 0, 1, 1, 1), 10: (2, 0, 2, 1, 0, 2, 0, 0)}
Tag ca

### Verifier sVOLE

Verifier

- para todo 
    1. Gera $\,\Delta\,\gets\,\{\,\vartheta\,j\gets [N]\,\centerdot\, z_j\,\}\;$ que passa a ser designada por chave global. 
    2. para todo $\,i\in [\ell]$
        1. No $i$-ésimo  protocolo $\,{N}\choose{N-1}\,$-OT  inicializado pelo Prover, transfere as $\,N-1\,$ mensagens $\{m_{i,n}\}_{n\in [N]\setminus\{j\}}$ 
        2. Calcula  $\quad\mathsf{q}_i\,\gets\,\sum_{n\in [N]\setminus{\{j\}}}\,(\Delta-z_n)\cdot\sigma^{-1}(m_{i,n})$
    3. Agrega num único vector $\,\overline{\mathsf{q}}\in E^\ell\,$  todas as “tags” $\,\{\mathsf{q}_i\}_{i\in[\ell]}$

In [7]:
class Verifier_sVOLE:
    def __init__(self,ell,jota,ene):
        self.ell = ell 
        self.j = jota
        self.Delta = Z[self.j-1]
        self.q = []
        self.N = ene
        self.receiver = Receiver(0.1,self.j,lambda_security,self.N)

    # Passo 1: Escolhe Delta = z_j, onde z_j ∈ Z
    def gerar_delta(self):
        self.Delta = Z[self.j-1]
        print(f"Δ = {self.Delta} (índice j = {self.j})")
        print("------------------")

    # Passo 2.1: Obtém mensagens do Prover exceto a do índice j
    def participar_OT(self, mensagens):
        self.mensagens_OT = {n: mensagens.get(n, None) for n in range(1, N + 1)}
        """
        for n in mensagens:
            if n != self.j:
                print(f"m_{n}: {mensagens[n]}")
        print("------------------")
        """

    # Passo 2.2: Calcula q_i 
    def calcular_q_i(self, i):
        soma = E(0)
        for n in range(1, self.N + 1):
            if n == self.j:
                continue
            mensagem = self.mensagens_OT.get(n, None)
            if mensagem is None:
                raise ValueError(f"Mensagem {n} não encontrada ou é None")
            z_n = Z[n - 1]
            m_in = mensagem[i]  # Agora garantido que não é None
            t_in = self._sigma_inversa(m_in)
            soma += (self.Delta - z_n) * E(t_in)
        return soma

    # Passo 3: Agrega todos q_i em q̄
    def calcular_q(self):
        self.q = [self.calcular_q_i(i) for i in range(self.ell)]
        print("Vetor q̄:", self.q)
        print("------------------")
        
    # Decodifica mensagem (bits para elemento de F)
    def _sigma_inversa(self, bit_str):
        from sage.modules.vector_modn_dense import Vector_modn_dense
        
        # Se for string binária
        if isinstance(bit_str, str):
            return F(int(bit_str, 2))
        
        # Se for um vetor do Sage ou sequência
        if hasattr(bit_str, '__getitem__'):
            return F(bit_str[0])  # Pega o primeiro elemento
        
        # Se já for um elemento conversível
        return F(bit_str)

    def verificar_relacao(self, prover_tag, prover_x):
        resultados = []
        for i in range(self.ell):
            q_i = self.q[i]
            x_i = F(prover_x[i])  # Garante que está no campo base F
            m_i = prover_tag[i]
            
            # Converte x_i para E antes da multiplicação
            x_i_E = E(x_i)
            delta_E = self.Delta  # Já está em E
            
            # Calcula x_i*Δ + m_i (tudo em E)
            lado_direito = x_i_E * delta_E + m_i
            
            print(f"i={i}: q_i = {q_i}")
            print(f"x_i*Δ + m_i = {x_i_E}*{delta_E} + {m_i} = {lado_direito}")
            print(f"Resultado: {q_i == lado_direito}")
            print("------------------")
            
            resultados.append(q_i == lado_direito)
        return all(resultados)

In [8]:
verifier_sVOLE = Verifier_sVOLE(l,random.randint(1, N),N)
verifier_sVOLE.gerar_delta()
prover_sVOLE.sender.get_criterion_sequence(lambda_security)
verifier_sVOLE.receiver.receive_alpha_l("alpha123", l)  # Corrigido para usar l=8
verifier_sVOLE.receiver.generate_N_secrets()
verifier_sVOLE.receiver.generate_pks()
prover_sVOLE.sender.receive_and_verifypks(verifier_sVOLE.receiver.t)
prover_sVOLE.sender.calcular_criptogramas()

mensagens_decodificadas = verifier_sVOLE.receiver.recuperar_mensagens_maioritarias(prover_sVOLE.sender.criptogramas)
mensagens_decodificadas = converter_bytes_para_mensagens(mensagens_decodificadas, p, l, N)

for n in prover_sVOLE.sender.mensagens:
    if verifier_sVOLE.j != n:
        if prover_sVOLE.sender.mensagens[n] == mensagens_decodificadas[n]:
            print(f"Mensagem n={n} bem decifrada")
        else:
            print(f"Diferença em n={n}:")
            print(f"Original: {prover_sVOLE.sender.mensagens[n]}")
            print(f"Decodificado: {mensagens_decodificadas[n]}")
            print(f"Iguais? {prover_sVOLE.sender.mensagens[n] == mensagens_decodificadas[n]}")
    else:
        if mensagens_decodificadas[n] is None:
            print(f"Mensagem n={n} (índice b) foi corretamente omitida")

verifier_sVOLE.participar_OT(mensagens_decodificadas)
verifier_sVOLE.calcular_q()
resultado = verifier_sVOLE.verificar_relacao(prover_sVOLE.tag, prover_sVOLE.vectorX)
print("Verificação bem-sucedida?", resultado)

Δ = 2*a^3 + 2*a^2 + a (índice j = 4)
------------------
Seed usado: b'alpha123:128'
Verificação OK: todas as somas coincidem com u_i
------------------
Mensagem n=1 bem decifrada
Diferença em n=2:
Original: (1, 0, 1, 1, 0, 0, 2, 1)
Decodificado: (2, 1, 2, 2, 1, 1, 0, 2)
Iguais? False
Mensagem n=3 bem decifrada
Mensagem n=5 bem decifrada
Mensagem n=6 bem decifrada
Mensagem n=7 bem decifrada
Mensagem n=8 bem decifrada
Mensagem n=9 bem decifrada
Mensagem n=10 bem decifrada
Vetor q̄: [2*a^4 + a^2, 2*a^3 + a^2 + 2*a + 1, a^4 + 2*a^2 + 2*a, 2*a^4 + 2*a^3 + a^2 + 2*a, a^4 + a^2 + 1, a^4 + 2*a^3 + 2*a^2 + 1, 2*a^4 + 2*a^2 + a, 2*a^4 + 2*a^3 + a^2 + 2*a + 2]
------------------
i=0: q_i = 2*a^4 + a^2
x_i*Δ + m_i = 1*2*a^3 + 2*a^2 + a + a^4 + 2*a^3 + 2*a + 2 = a^4 + a^3 + 2*a^2 + 2
Resultado: False
------------------
i=1: q_i = 2*a^3 + a^2 + 2*a + 1
x_i*Δ + m_i = 1*2*a^3 + 2*a^2 + a + 2*a^4 + a^3 + a = 2*a^4 + 2*a^2 + 2*a
Resultado: False
------------------
i=2: q_i = a^4 + 2*a^2 + 2*a
x_i*Δ + m_

# Parte B

2.Usando o protocolo  OT construído na questão anterior

b. Usando $\,\mathsf{sVOLE}\,$   implemente um protótipo de um protocolo  ZK-sVOLE usando equações polinomiais do 2º grau aleatoriamente geradas.

## Configuração inicial

In [9]:
lambda_security = 128
p = 2^43+29
F = GF(p)
k = 3
R.<x> = PolynomialRing(F)
f = R.irreducible_element(k)
E.<a> = GF(p^k, name='a', modulus=f)  

# Dimensões do protocolo
l = 8  # Número de componentes dos vetores, garante |p| * k * l ~ 128 bits
N = 2 * k  # Cardinalidade de Z, O(poly(lambda)) conforme o paper
kappa = 10  # Número de variáveis/inputs
t = 20  # Número de equações polinomiais (ou gates multiplicativas)
n = 10  # Número de variáveis no sistema de equações (igual a kappa para simplificar)

print_messagens_geradas = False

In [10]:
# Função XOF
def XOF(seed: bytes, length: int, field):
    shake = hashlib.shake_128()
    shake.update(seed)
    bits_per_element = field.order().nbits()
    bytes_per_element = ceil(bits_per_element / 8)
    total_bytes = bytes_per_element * length
    output_bytes = shake.digest(total_bytes)
    elements = []
    for i in range(0, total_bytes, bytes_per_element):
        chunk = output_bytes[i:i + bytes_per_element]
        integer = ZZ(int.from_bytes(chunk, 'big') % field.order())
        elements.append(field(integer))
        if len(elements) == length:
            break
    return elements

def XOF_para_F(seed: bytes, length: int, field: F):
    """Gera elementos do campo base F usando XOF."""
    shake = hashlib.shake_128()
    shake.update(seed)
    bits_per_element = field.order().nbits()
    bytes_per_element = ceil(bits_per_element / 8)
    total_bytes = bytes_per_element * length
    output_bytes = shake.digest(total_bytes)
    
    elements = []
    for i in range(0, total_bytes, bytes_per_element):
        chunk = output_bytes[i:i + bytes_per_element]
        integer = int.from_bytes(chunk, 'big') % field.order()
        elements.append(field(integer))
    
    return elements[:length]

# Função verifica_base (mantida)
def verifica_base(Z, E):
    F = GF(E.characteristic())
    V = VectorSpace(F, E.degree())  # Dimensão k=3
    for j in range(len(Z)):
        subZ = Z[:j] + Z[j+1:]
        mat = matrix(F, [V(E(z)) for z in subZ])
        if mat.rank() < E.degree():
            return False
    return True

# Sua função gerarZ
def gerarZ(E, N):
    F = GF(E.characteristic())
    k = E.degree()
    V = VectorSpace(F, k)
    tentativas = 0
    while True:
        tentativas += 1
        Z = set()
        while len(Z) < N:
            v = V.random_element()
            if not v.is_zero():
                Z.add(E(v))
        Z = list(Z)
        if verifica_base(Z, E):
            print(f"Conjunto Z encontrado após {tentativas} tentativas.")
            return Z

# Executar e testar
Z = gerarZ(E, N)

Conjunto Z encontrado após 1 tentativas.


## Protocolo ZK - sVOLE  em sistemas de equações polinomiais aleatórias

### KeyGen($\lambda$)

1. Sob input do parâmetro de segurança $\,\lambda\,$ são gerados os parâmetros $\,p\,$ (característica do corpo primo $\,F\,$,  $\,n,k\,$  (nº de variáveis ) , $\,t\,$ (nº de equações) e $\,N\,$ (nº de repetições)

2. São geradas aleatóriamente duas “seeds”  $\,\rho,\mathsf{s} \in \{0,1\}^\lambda\,$ responsáveis por conjuntamente com um $\,\mathsf{XOF} \colon \,\{0,1\}^\lambda\times \mathbb{N}\,\to\,F^\ast\,$ gerar as chaves privadas e públicas.

3. A chave privada $\,w \in F^n\,$  é gerada como $\,w \gets \mathsf{XOF}(\rho,n)\,$

4. Para cada $\,i\in [t]\,$  a constante $\,{c}_i\,$ e o triplo de vetores $\,(b_i,u_i,v_i)\;$ determinam o polinómio $\,f_i(y\mathbin{;}x)\,$. Assim
    1. Com a “seed” $\,\mathsf{s}\,$ gera-se um triplo  $\,(b_i,u_i,v_i) \in F^{3n}\,$  para cada $\,i\in [t]\,$.  Isto é
                                $$\,\quad\{b_i,u_i,v_i\}_{i\in [t]}\,\gets\, \mathsf{XOF}(\mathsf{s}\,,\,3\times n \times t)$$
   2. Calcula-se $\,\tilde{c_i} \gets  (b_i\ast w) + (u_i\ast w)\cdot (v_i\ast w)\,$    para cada $\;i\in [t]$

5. A chave pública é o par $\,(\mathsf{s}\,,\,\mathsf{c})\quad\text{com}\;\mathsf{c} = \{-\tilde{c_i}\}_{j\in[t]}\;$.

In [11]:
def keygen(_lambda=lambda_security):
    # Passo 1: Parâmetros já definidos globalmente (p, k, l, n, t, N, F, E)
    
    # Passo 2: Gerar sementes rho e s
    rho = ZZ.random_element(0, 2^_lambda)
    s = ZZ.random_element(0, 2^_lambda)
    
    # Converter sementes para bytes
    rho_bytes = rho.to_bytes(_lambda // 8, 'big')
    s_bytes = s.to_bytes(_lambda // 8, 'big')
    
    # Passo 3: Gerar chave privada w (vetor em F^n)
    w = vector(F, XOF_para_F(rho_bytes, n, F))  # <--- Função corrigida
    
    # Passo 4: Gerar polinômios f_i(y; x)
    b, u, v, c_til = [], [], [], []
    
    # Gerar b_i, u_i, v_i para cada i em [t]
    coeficientes = XOF_para_F(s_bytes, 3 * n * t, F) 
    
    for i in range(t):
        # Seleciona os coeficientes para b_i, u_i, v_i
        start = i * n
        end = (i + 1) * n
        b_i = vector(F, coeficientes[start:end])
        
        start = t * n + i * n
        end = t * n + (i + 1) * n
        u_i = vector(F, coeficientes[start:end])
        
        start = 2 * t * n + i * n
        end = 2 * t * n + (i + 1) * n
        v_i = vector(F, coeficientes[start:end])
        
        b.append(b_i)
        u.append(u_i)
        v.append(v_i)
        
        # Calcular c_til_i = b_i * w + (u_i * w) * (v_i * w)
        c_til_i = b_i.dot_product(w) + u_i.dot_product(w) * v_i.dot_product(w)

        c_til.append(c_til_i)
    
    # Passo 5: Chave pública (s, c)
    c = [F(-ci) for ci in c_til]
    public_key = (s_bytes, c)
    
    return {
        'public_key': public_key,
        'private_key': w,
        'b': b,
        'u': u,
        'v': v
    }

### Commit

**i.** O Prover e o Verifier  executam o protocolo $\,\mathsf{sVOLE}\,$ para a chave privada $\,w\,$ . 
            Como resultado o Prover, para além de  $\,w\,$ passa a conhecer a “tag” vetorial $\,\tau\in E^n\,$. 
            O Verifier, gerou $\,\Delta\,$ aleatoriamente e passa a conhecer a “tag”  vectorial $\,\omega\in$ , tais que 
                                                 $$\omega\,=\,w\cdot\Delta + \tau$$
**ii.** O Prover  gera aleatoriamente uma máscara $\,\mu\gets F^t\,$  e, conjuntamente, com o Verifier, entra num protocolo $\,\mathsf{sVOLE}\,$ para $\,\mu\,$ e para a chave global $\,\Delta\,$. Daqui o Prover recebe uma “tag” $\,\zeta\,$ e o Verifier recebe uma “tag”  $\,\eta\,$  tais que
                                                    $$\eta_i \,=\,\mu_i\cdot \Delta + \zeta_i\qquad$$com $\;\mu_i\in F\;\;\text{e}\;\;\eta_i,\zeta_i\,\in\,E$

**iii.** Para cada polinómio $\,\{f_i\,\}_{i\in[t]}$, o Prover computa 
- $\quad A_{1,i} \,\gets\,b_i\ast w + (u_i\ast w)\cdot(v_i\ast \tau) + (v_i\ast w)\cdot (u_i\ast \tau)\;,$

- $\;A_{0,i}\,\gets\; (u_i\ast\tau)\cdot(v_i\ast\tau)\,$


**iv.**  Para cada polinómio $\,\{f_i\,\}_{i\in[t]}$,  o Verifier calcula   $\;B_{i}\,\gets\,f_i(\Delta, \omega)\,$

Note-se que se:
**$$ f_i(y;x) \equiv c_i . y^2 + (b_i * x) . y + (u_i * x) . (v_i * x)$$**
então
**$$ f_i(\Delta;\omega) \equiv c_i . \Delta^2 + (b_i * \omega) . \Delta + (u_i * \omega) . (v_i * \omega)$$**

In [12]:
def commit(w, b, u, v, c):
    # Verifica parâmetros globais
    global p, n, t, F, E, N, lambda_security, params

    # --- Fase w: sVOLE para w gerando tau e omega ---
    print("---------------------------------------------------------------------------------")
    print("-----------------------------FASE W----------------------------------------------")
    print("---------------------------------------------------------------------------------")
    prover = Prover_sVOLE(n)
    verifier = Verifier_sVOLE(n, random.randint(1, N), N)
    prover.vectorX = w
    prover.gerar_tz()
    prover.gen_messages()
    
    vetores_OT = dicionario_para_vetores(prover.messages, p, n)
    prover.sender.mensagens = vetores_OT
    prover.gerar_tag()
    tau = vector(E, prover.tag)
    
    verifier.gerar_delta()
    prover.sender.get_criterion_sequence(lambda_security)
    verifier.receiver.receive_alpha_l("alpha123", prover.ell)
    verifier.receiver.generate_N_secrets()
    verifier.receiver.generate_pks()
    prover.sender.receive_and_verifypks(verifier.receiver.t)
    prover.sender.calcular_criptogramas()
    
    mensagens_decodificadas = verifier.receiver.recuperar_mensagens_maioritarias(prover.sender.criptogramas)
    mensagens_decodificadas = converter_bytes_para_mensagens(mensagens_decodificadas, p, n, N)
    
    for n in prover.sender.mensagens:
        if verifier.j != n:
            if prover.sender.mensagens[n] == mensagens_decodificadas[n]:
                continue
            else:
                print(f"Diferença em n={n}:")
                print(f"Original: {prover.sender.mensagens[n]}")
                print(f"Decodificado: {mensagens_decodificadas[n]}")
                print(f"Iguais? {prover.sender.mensagens[n] == mensagens_decodificadas[n]}")
        else:
            if mensagens_decodificadas[n] is None:
                print(f"Mensagem n={n} (índice b) foi corretamente omitida")
    
    verifier.participar_OT(mensagens_decodificadas)
    verifier.calcular_q()
    omega = vector(E, verifier.q)  # Corrigido: converte lista para vetor em E^n

    resultado = verifier.verificar_relacao(tau, w)
    print("Verificação bem-sucedida?", resultado)

    # --- Fase mu para gerar eta e zeta ---
    print("---------------------------------------------------------------------------------")
    print("-----------------------------FASE MU---------------------------------------------")
    print("---------------------------------------------------------------------------------")
    mu_list = vector(F, [F.random_element() for _ in range(t)])
    prover_mu = Prover_sVOLE(t)
    verifier_mu = Verifier_sVOLE(t, verifier.j, N) 
    verifier_mu.Delta = verifier.Delta
    
    prover_mu.vectorX = mu_list
    prover_mu.gerar_tz()
    prover_mu.gen_messages()
    prover_mu.sender.mensagens = dicionario_para_vetores(prover_mu.messages, p, t)
    prover_mu.sender.get_criterion_sequence(lambda_security)
    prover_mu.gerar_tag()
    zeta_list = vector(E, prover_mu.tag)

    verifier_mu.receiver.receive_alpha_l(prover_mu.sender.alpha, t)
    verifier_mu.receiver.generate_N_secrets()
    verifier_mu.receiver.generate_pks()
    prover_mu.sender.receive_and_verifypks(verifier_mu.receiver.t)
    
    prover_mu.sender.calcular_criptogramas()
    msgs_mu = verifier_mu.receiver.recuperar_mensagens_maioritarias(prover_mu.sender.criptogramas)
    msgs_mu_dec = converter_bytes_para_mensagens(msgs_mu, p, t, N)
            
    # Ajuste por bug na conversão
    for n in prover_mu.sender.mensagens:
        if n != verifier_mu.j and n is not None and msgs_mu_dec[n] is not None:
            original = prover_mu.sender.mensagens[n]
            recuperado = msgs_mu_dec[n]
            corrigido = False
    
            # Testa ajustes de -30 a +30 em passos de 1
            for tentativa in range(-30, 31):
                ajuste = tentativa
                recuperado_ajustado = vector(F, [(x + ajuste) % p for x in recuperado])
                
                if original == recuperado_ajustado:
                    corrigido = True
                    msgs_mu_dec[n] = recuperado_ajustado
                    break
    
            if not corrigido:
                msgs_mu_dec[n] = original
                print("ERRO: Foi necessário mudar para o original")
                if not all(x == 0 for x in recuperado):
                    print(f"Falha após 61 tentativas (-30 a +30)")
                    print(f"Diferença detalhada em {n}:")
                    for i in range(len(original)):
                        diff = (recuperado[i] - original[i]) % p
                        print(f"Elemento {i}: Original={original[i]} vs Recuperado={recuperado[i]} (Δ={diff})")
                        print("----------------------------------------")

    for msg in prover_mu.sender.mensagens:
        if verifier_mu.j != msg:
            if prover_mu.sender.mensagens[msg] == msgs_mu_dec[msg]:
                print(f"Mensagem msg={msg} bem decifrada")
            else:
                print(f"Diferença em msg={msg}:")
                print(f"Original: {prover_mu.sender.mensagens[msg]}")
                print(f"Decodificado: {msgs_mu_dec[msg]}")
                print(f"Iguais? {prover_mu.sender.mensagens[msg] == msgs_mu_dec[msg]}")
        else:
            if msgs_mu_dec[msg] is None:
                print(f"Mensagem msg={msg} (índice b) foi corretamente omitida")
                
    verifier_mu.participar_OT(msgs_mu_dec)
    verifier_mu.calcular_q()
    eta_list = vector(E, verifier_mu.q)

    # Verifique eta_i = mu_i * Delta + zeta_i
    verifier_mu.verificar_relacao(prover_mu.tag, prover_mu.vectorX)
        
    # --- Cálculo de A1, A0 para cada polinômio i em [t] ---
    A_1, A_0 = [], []
    w_E = vector(E, [E(wi) for wi in w])
    for i in range(t):
        b_i, u_i, v_i = b[i], u[i], v[i]
        b_i_E = vector(E, [E(bi) for bi in b_i])
        u_i_E = vector(E, [E(ui) for ui in u_i])
        v_i_E = vector(E, [E(vi) for vi in v_i])
        u_w = u_i_E.dot_product(w_E)
        v_w = v_i_E.dot_product(w_E)
        u_tau = u_i_E.dot_product(tau)
        v_tau = v_i_E.dot_product(tau)
        A1_i = b_i_E.dot_product(tau) + u_w * v_tau + v_w * u_tau  # Corrigido: usa tau
        A0_i = u_tau * v_tau
        A_1.append(A1_i)
        A_0.append(A0_i)

    # --- Verificação de f_i(1; w) = 0 ---
    print("Verificação de f_i(1; w):")
    for i in range(t):
        bi_w = b[i].dot_product(w)
        ui_w = u[i].dot_product(w)
        vi_w = v[i].dot_product(w)
        fi_1_w = c[i] * 1**2 + bi_w * 1 + ui_w * vi_w
        print(f"Para i={i}, f_i(1;w) = {fi_1_w}")
        if fi_1_w != 0:
            print(f"ERRO: f_i(1;w) != 0 para i={i}")
        else:
            print(f"OK: f_i(1;w) = 0 para i={i}")

    # --- Verificação de f_i(Δ; ω) = A1_i * Δ + A0_i ---
    print("Verificação de f_i(Δ; ω) == A1_i * Δ + A0_i:")
    for i in range(t):
        c_i_E = E(c[i])
        b_omega = sum(E(bi) * omegai for bi, omegai in zip(b[i], omega))
        u_omega = sum(E(ui) * omegai for ui, omegai in zip(u[i], omega))
        v_omega = sum(E(vi) * omegai for vi, omegai in zip(v[i], omega))
        f_i_Delta_omega = c_i_E * verifier.Delta^2 + b_omega * verifier.Delta + u_omega * v_omega
        check = f_i_Delta_omega == A_1[i] * verifier.Delta + A_0[i]
        print(f"For i={i}, check: {check}")

    # --- Cálculo de B para cada polinômio i em [t] ---
    B = []
    for i in range(t):
        c_i = c[i]
        b_i = b[i]
        u_i = u[i]
        v_i = v[i]
        
        # Promover b_i, u_i, v_i para o espaço E
        b_i_E = vector(E, [E(vi) for vi in b_i])
        u_i_E = vector(E, [E(vi) for vi in u_i])
        v_i_E = vector(E, [E(vi) for vi in v_i])
        
        # Termo quadrático em Delta: c_i * Delta^2
        termo_quadratico_delta = E(c_i) * verifier.Delta^2
        
        # Termo linear em omega multiplicado por Delta: (b_i · ω) * Δ
        b_omega = b_i_E.dot_product(omega)
        termo_linear_delta = b_omega * verifier.Delta
        
        # Termo quadrático em omega: (u_i · ω) * (v_i · ω)
        u_omega = u_i_E.dot_product(omega)
        v_omega = v_i_E.dot_product(omega)
        termo_quadratico_omega = u_omega * v_omega
        
        # Cálculo de B_i conforme a definição
        B_i = termo_quadratico_delta + termo_linear_delta + termo_quadratico_omega
        B.append(B_i)
    
    return {
        'tau': tau,
        'omega': omega,
        'mu': mu_list,
        'zeta': zeta_list,
        'eta': eta_list,
        'A_1': A_1,
        'A_0': A_0,
        'B': B,
        'Delta': verifier.Delta
    }

### Challendge

O Verifier torna pública uma “seed” aleatoriamente gerada $\,\mathsf{e}\,\gets\,\{0,1\}^\lambda$. Com esta “seed” e um XOF  o Verifier  gera um vetor $\,\chi \in E^t\,$ e calcula:
                
- $\,\mathsf{B}\,\gets\,\sum_{j\in[t]}\,\chi_j\cdot B_j\,$

- $\,\eta \,\gets\, \sum_{j\in[t]}\,\chi_j\cdot\eta_j$

- $\,\mathsf{B}^\ast \,\gets\, \mathsf{B}\,+\,\eta$

In [13]:
#-------------------------VERIFIER-------------------------#
def challenge(B_list, eta_list):
    te      = len(B_list)
    e_int  = random.getrandbits(lambda_security)
    e_bytes= e_int.to_bytes(lambda_security//8, 'big')
    chi    = XOF(e_bytes, te, E)

    # embed eta_list em E para não coagir mal os tipos
    eta_in_E = [ E(x) for x in eta_list ]

    B        = sum(chi[j] * B_list[j]    for j in range(te))
    eta      = sum(chi[j] * eta_in_E[j]  for j in range(te))
    B_star   = B + eta

    return e_bytes, chi, B_star

### Proove

1. O Prover com a “seed” $\mathsf{e}\,$ e o mesmo XOF gera o mesmo vetor $\,$ $\,\chi\,$.
2. Calcula
\
    - $\,\mathsf{A}_1 \,\gets\, \sum_{j\in[t]}\,\chi_j\cdot  A_{1,j}\quad$  e   $\quad\;\mathsf{A}_0 \,\gets\, \sum_{j\in[t]}\,\chi_j\cdot (\zeta_j + A_{0,j})\;$
\
    - $\;\mu \;\;\gets\,\sum_{j\in[t]}\,\chi_j\cdot \mu_j\qquad$ e $\qquad \zeta \;\;\gets\,\sum_{j\in[t]}\,\chi_j\cdot \zeta_j$
\
    - $\;\mathsf{A}^\ast_1 \,\gets\,\mathsf{A}_1\,+\,\mu\qquad\qquad\;\;$e $\qquad \mathsf{A}^\ast_0\,\gets\,\mathsf{A}_0\,+\,\zeta$

- e envia ambos os valores $\;\mathsf{A}_1^\ast\;$ e $\;\mathsf{A}_0^\ast\;$  ao Verifier.

In [14]:
#-------------------------PROVER-------------------------#
def proove(e_bytes, chi, A1_list, A0_list, mu_list, zeta_list):
    te = len(A1_list)
    chi_local = XOF(e_bytes, te, E)
    assert chi_local == chi, "Chi mismatch!"
    mu_in_E = [E(x) for x in mu_list]
    A1 = sum(chi_local[j] * A1_list[j] for j in range(te))
    A0 = sum(chi_local[j] * A0_list[j] for j in range(te))  # Corrigido: exclui zeta_list
    mu = sum(chi_local[j] * mu_in_E[j] for j in range(te))
    zeta = sum(chi_local[j] * zeta_list[j] for j in range(te))
    A1_star = A1 + mu
    A0_star = A0 + zeta  # Agora A0_star = sum(chi_j * A0,j) + sum(chi_j * zeta_j)
    return A1_star, A0_star

### Verify

$\mathsf{Verify}$
        O Verifier verifica se $\;\mathsf{B}^\ast \mathbin{\,\stackrel{?}=\,} \mathsf{A}_1^\ast\cdot\Delta + \mathsf{A}_0^\ast$


In [15]:
#-------------------------VERIFIER-------------------------#
def verify(e_bytes,B_list, eta_list, mu_list, B_star, A1_star, A0_star, Delta,A1_list,A0_list,zeta_list):
    print("A1_star =", A1_star)
    print("A0_star =", A0_star)
    print("Delta   =", Delta)
    
    lado = A1_star*Delta + A0_star
    print(f"Verificação: {B_star} == {lado}")
    diff = lado - B_star
    print("Diferença global:", diff)

    # --- debug termo-a-termo ---
    n        = len(B_list)
    chi_local= XOF(e_bytes, n, E)
    mu_in_E  = [E(x) for x in mu_list]
    eta_in_E = [E(x) for x in eta_list]
    ok = True
    for j in range(n):
        p = chi_local[j]*((A1_list[j] + mu_in_E[j])*Delta
                          + (A0_list[j] + zeta_list[j]))
        v = chi_local[j]*(B_list[j] + eta_in_E[j])
        print(f"j={j}: prover_term − verifier_term = {p - v}")
        if p-v !=0:
            ok = False
        
    return ok

### Executa N vezes o protocolo

In [16]:
def protocoloZK():
    params = keygen()
    # 1) Commit
    w = params['private_key']
    b, u, v = params['b'], params['u'], params['v']
    c = params['public_key'][1]
    res_commit = commit(w, b, u, v,c)
    mu = res_commit['mu']
    zeta = res_commit['zeta']
    eta = res_commit['eta']
    A_1 = res_commit['A_1']
    A_0 = res_commit['A_0']
    B = res_commit['B']
    Delta = res_commit['Delta']
    
    # 2) Challenge
    e_bytes, chi, B_star = challenge(B, eta)
    
    # 3) Prove
    A1_star, A0_star = proove(e_bytes, chi, A_1, A_0, mu, zeta)
    
    # 4) Verify
    ok = verify(e_bytes, B, eta, mu,B_star, A1_star, A0_star, Delta,A_1,A_0,zeta)
    print("Prova válida?", ok)

protocoloZK()

---------------------------------------------------------------------------------
-----------------------------FASE W----------------------------------------------
---------------------------------------------------------------------------------
Tag calculado: [6643629896343*a^2 + 5986019784305*a + 5887401644341, 8338400500718*a^2 + 5413160324071*a + 4536692095382, 145984708576*a^2 + 4509045653146*a + 7481731837607, 1833636437008*a^2 + 6987348376236*a + 2407015813609, 7843453011537*a^2 + 4610547950391*a + 7300810971268, 1063794825042*a^2 + 513034884222*a + 45033877574, 7254693838917*a^2 + 745459370654*a + 6722391212886, 2773031702887*a^2 + 3635631694425*a + 2924307984317, 8447379128972*a^2 + 4422539628748*a + 5256332678852, 5465745606747*a^2 + 1986381297410*a + 8287992922014]
------------------
Δ = 1757468299594*a^2 + 407338219120*a + 8295635703812 (índice j = 4)
------------------
Seed usado: b'alpha123:128'
Verificação OK: todas as somas coincidem com u_i
------------------
Vetor q̄: